# Exploration of the SWaT dataset

###  Where does this data come from:
1. I went [here](https://itrust.sutd.edu.sg/itrust-labs_datasets/) and requested the WADI and SWaT datasets. 
2. I only downloaded the 2015 folder from the Google drive folder they granted me access to. This gave me lots of zip files though.
3. I copied the zip files, ran something like:

```sh
mkdir all
for x in *.zip ; do unzip -d all -o -u $x ; done
```
4. Since the "Network" folder was huge I decided to include the tables in the "Physical" folder only for now.


In [ ]:
import os
os.chdir('..')

In [ ]:
import pandas as pd
import diag_vae.constants as const
# from pandas_profiling import ProfileReport
from datetime import datetime
import matplotlib.pyplot as plt
from ipywidgets import interact
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Read the data

In [ ]:
# reading the time series data files (physics only!)
raw_file_paths = [const.SWAT_RAW_NORMAL_V0_PATH, const.SWAT_RAW_NORMAL_V1_PATH, const.SWAT_RAW_ATTACK_V0_PATH]
proc_file_paths = [const.SWAT_PARQUET_NORMAL_V0_PATH, const.SWAT_PARQUET_NORMAL_V1_PATH, const.SWAT_PARQUET_ATTACK_V0_PATH]



In [ ]:
for raw_file_path, proc_file_path in zip(raw_file_paths, proc_file_paths):
   if not os.path.exists(proc_file_path):
      df =15
      df.to_parquet(proc_file_path)

In [ ]:
df_normal_v0, df_normal_v1, df_attack_v0 = [pd.read_parquet(path) for path in proc_file_paths]

# read labels file
df_label = pd.read_excel(const.SWAT_RAW_LABEL_PATH)

## Check heads and basic stats

In [ ]:
df_attack_v0.head()

In [ ]:
df_normal_v1.head()

In [ ]:
df_attack_v0.head()

## Symbols MAP

![SWAT Comps](../figures/SWAT_Components.png)

In [ ]:
df_normal_v1.columns

In [ ]:
all_sensor_cols = 

In [ ]:
{f'Comp_{i}':[c for c in df_normal_v1.columns if f'{i}0' in c] for i in range(1,7)}

In [ ]:
df_normal_v0['Normal/Attack'].unique()

In [ ]:
df_label

## What is the difference between those two files?

In [ ]:
# Timeframe? Takes a minute or two
for ind, df in enumerate([df_normal_v0, df_normal_v1]):
    print(f'Start timestamp v{ind}: {pd.to_datetime(df[" Timestamp"]).min()}')
    print(f'End timestamp v{ind}: {pd.to_datetime(df[" Timestamp"]).max()}')

--> v1 seams to start half an hour later. I read somewhere that that is on purpose
but can't find where anymore. 

In [ ]:
# Same schema in v0 and v1?
all(df_normal_v0.columns == df_normal_v1.columns)

--> Let's stick to v1, they'll have a reason for the
version bump anyways.

In [ ]:
# Same schema in v1 and attack?
all(df_normal_v0.columns == df_attack_v0.columns)

In [ ]:
set(df_attack_v0.columns).difference(set(df_normal_v1.columns))

Haha, they messed up the column names... I'll fix that later!

## Frequency?

In [ ]:
pd.to_datetime(df_normal_v0[' Timestamp']).diff()[1:].unique()

Ok, seems to be one seconds between all the timestamps

## Basic stats

In [ ]:
df_normal_v0.info()

No missing values, nice!

In [ ]:
# profile = ProfileReport(df_normal_v0,
#                         minimal=True, title='Pandas Profiling Report',
#                         explorative=True)
# profile.to_widgets()

## Check the label table

In [ ]:
# check label dataset
df_label

There seams to be a bug in the labels data set. There are entries for jan 2015. For that reason we filter the data frames according to a min and max date in the config file (constants.py)

In [ ]:
# filter labels df to the attack that have a end date attached
# transofrm end time to full timestmap
df_label_time = df_label[df_label['End Time'].notna()].copy()
df_label_time.loc[:, 'End Time'] = [datetime.combine(datetime.date(a), b) for a,b in zip(
    df_label_time['Start Time'], df_label_time['End Time'])]
df_label_time = df_label_time.reset_index(drop=True)

In [ ]:
# plot timestamp for attach df
plt.plot(df_attack_v0.index.values)

In [ ]:
# plot timestamps of labels
plt.plot(df_label_time['Start Time'].values)

## Lets plot som time series

In [ ]:
# clean time stamp colname
for df in [df_normal_v1, df_attack_v0]:
    df['Timestamp'] = pd.to_datetime(df[' Timestamp'])

In [ ]:
# convert to pandas time series
df_normal_v1 = df_normal_v1.set_index('Timestamp', drop=True)
df_attack_v0 = df_attack_v0.set_index('Timestamp', drop=True)
df_normal_v1 = df_normal_v1.drop(' Timestamp', axis=1)
df_attack_v0 = df_attack_v0.drop(' Timestamp', axis=1)

In [ ]:
# fix column names (some begin with white spaces)
df_normal_v1.columns = [s.replace(' ', '') for s in df_normal_v1.columns]
df_attack_v0.columns = [s.replace(' ', '') for s in df_attack_v0.columns]

In [ ]:
# ok, lets remove everything smaller than min_date and larger than max date from the attacks and labels
df_label_time = df_label_time[(df_label_time['Start Time'] > const.SWAT_MIN_DATE) &
                         (df_label_time['Start Time'] < const.SWAT_MAX_DATE)]
df_attack_v0 = df_attack_v0[(df_attack_v0.index > const.SWAT_MIN_DATE) &
                         (df_attack_v0.index < const.SWAT_MAX_DATE)]

In [ ]:
df_normal_v1.head()

In [ ]:
df_label_time['Start Time'].min()

In [ ]:
# define plot function
def get_4_signal_plot(sensor0='AIT503', sensor1='AIT203',
                      sensor2='FIT501', sensor3='LIT401', day='all', sample_freq='60s', 
                     show_attacks=False):
    df_plot = df.copy()
    df_plot_label = df_label_time.copy()
    print(df_plot_label['Start Time'].min())
    if not day == 'all':
        df_plot = df_plot[(df_plot.index > pd.Timestamp(day)) &
                         (df_plot.index < pd.Timestamp(day)+pd.Timedelta('1d'))]
        df_plot_label = df_plot_label[(df_plot_label['Start Time'] > pd.Timestamp(day)) &
                         (df_plot_label['Start Time'] < pd.Timestamp(day)+pd.Timedelta('1d'))]
    df_plot = df_plot.resample(sample_freq).mean()

    
    sensor_selection_list=[sensor0, sensor1, sensor2, sensor3]
    num_plots = 5 if show_attacks else 4
    fig = make_subplots(rows=num_plots, cols=1, shared_xaxes=True)
    for i, s in enumerate(sensor_selection_list):
        fig.add_trace(
            go.Scatter(x=df_plot.index,
                       y=df_plot[s], name=s),
            row=i+1, col=1,
        )
    if show_attacks:
        for i in range(len(df_plot_label)):
            df_plot_label = df_plot_label.reset_index(drop=True)
            start = df_plot_label.loc[i, 'Start Time']
            end = df_plot_label.loc[i, 'End Time']
            attack = df_plot_label.loc[i, 'Attack #']
            fig.add_trace(
                go.Scatter(x=[start, end],
                           y=[1, 1], name=f'Attack #{attack}'),
                row=5, col=1,
            )

    title = f'{sensor0}, {sensor1}, {sensor2} and {sensor3}'\
            f' over time (averaged over {sample_freq})'

    fig.update_layout(height=800, width=800, title_text=title)
    fig.show()

In [ ]:
# define drop down lists (see plots below)
day_list_normal = [f'2015-12-{d}' for d in range(22,28)] + ['all']
day_list_attack = [f'2015-12-{d}' for d in range(28,31)] + ['2016-01-01'] + \
                  ['2016-02-01'] + ['all']
sample_frequency_list = ['1s', '5s', '10s', '60s']
df = df_attack_v0.copy()
sensors = list(df.columns)
sensors.remove('Normal/Attack')
sensors.sort()
interact(get_4_signal_plot, sensor0=sensors, sensor1=sensors,
         sensor2=sensors, sensor3=sensors, day=day_list_attack,
         sample_freq=sample_frequency_list, show_attacks=True)